Preparing the data for trainung/testing the model 

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np 
from sklearn import preprocessing

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Customer-Retention-data/train_final.csv')
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Customer-Retention-data/test_final.csv')

In [0]:
train_features = train.drop(['new.returningCustomer'],axis=1)
train_labels = train['new.returningCustomer']
test_features = test.drop(['new.returningCustomer'],axis=1)
test_labels = test['new.returningCustomer']


In [0]:
# Initialize label binarizer
lb = preprocessing.LabelBinarizer()

# Training data
train_features['trafficSource.isTrueDirect'] = lb.fit_transform(train_features['trafficSource.isTrueDirect'])
train_features['device.isMobile'] = lb.transform(train_features['device.isMobile'])
test_features['trafficSource.isTrueDirect'] = lb.fit_transform(test_features['trafficSource.isTrueDirect'])
test_features['device.isMobile'] = lb.transform(test_features['device.isMobile'])


In [7]:
for column in train_features.columns:
    if (train_features[column].dtype != np.int64 and train_features[column].dtype != np.float64):
        print("Converting column: {}".format(column))

        le = preprocessing.LabelEncoder()
        train_features[column] = le.fit_transform(train_features[column])
        del le
 
 
for column in test_features.columns:
    if (test_features[column].dtype != np.int64 and test_features[column].dtype != np.float64):
        print("Converting column: {}".format(column))

        le = preprocessing.LabelEncoder()
        test_features[column] = le.fit_transform(test_features[column])
        del le       

Converting column: channelGrouping
Converting column: device.browser
Converting column: device.deviceCategory
Converting column: device.operatingSystem
Converting column: geoNetwork.city
Converting column: geoNetwork.continent
Converting column: geoNetwork.country
Converting column: geoNetwork.networkDomain
Converting column: geoNetwork.region
Converting column: geoNetwork.subContinent
Converting column: trafficSource.adContent
Converting column: trafficSource.adwordsClickInfo.adNetworkType
Converting column: trafficSource.adwordsClickInfo.gclId
Converting column: trafficSource.adwordsClickInfo.slot
Converting column: trafficSource.campaign
Converting column: trafficSource.keyword
Converting column: trafficSource.medium
Converting column: trafficSource.referralPath
Converting column: trafficSource.source
Converting column: channelGrouping
Converting column: device.browser
Converting column: device.deviceCategory
Converting column: device.operatingSystem
Converting column: geoNetwork.ci

In [0]:
# Select subset of data to keep based on heatmap
keep_cols = ['totals.bounces',
             'totals.hits',
             'totals.newVisits',
             'totals.pageviews',
             'totals.sessionQualityDim',
             'totals.timeOnSite',
             'trafficSource.isTrueDirect',
             'visitNumber',
             'country_dem',
             'latitude',
             'longitude']
# Select subset of columns
train_feat = train_features[keep_cols]
test_feat = test_features[keep_cols]

In [9]:
# show the distribution
print('y_train class distribution')
unique_elements, counts_elements = np.unique(train_labels, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))
print('y_test class distribution')
unique_elements, counts_elements = np.unique(test_labels, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

print('Training Features Shape:', train_feat.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_feat.shape)
print('Testing Labels Shape:', test_labels.shape)

y_train class distribution
[[      0       1]
 [1024244  513259]]
y_test class distribution
[[     0      1]
 [219121 142308]]
Training Features Shape: (1537503, 11)
Training Labels Shape: (1537503,)
Testing Features Shape: (361429, 11)
Testing Labels Shape: (361429,)


In [0]:
train_feat = train_feat(1:155000,:)
train_labels = train_labels(1:155000,:)

print('Training Features Shape:', train_feat.shape)
print('Training Labels Shape:', train_labels.shape)

In [0]:
#Importing the two models 
import pickle
rf1 = pickle.load(open("/content/gdrive/My Drive/Colab Notebooks/Random_Forest_CR1.sav",'rb'))
xg1 = pickle.load(open("/content/gdrive/My Drive/Colab Notebooks/XGB_CR1.sav",'rb'))


In [0]:
#Voting Classifier 

from sklearn.ensemble import VotingClassifier#create a dictionary of our models
estimators=[("rf", rf1), ("xgb", xg1)]#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting= 'hard')

In [17]:
#fit model to training data
ensemble.fit(train_feat, train_labels)


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=300,
                                                     n_jobs=None,
          

In [18]:
#test our model on the test data
ensemble.score(test_feat, test_labels)

0.8637602406005053